In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"//data//"+p, data_list))[1:2] #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240731_0217.raw']


In [4]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
import matplotlib.pyplot as plt

interval = 1_00 # Set your monitoring chunck size
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 500 # End the loop early
    
for file in file_path:
    fReader = rawFileReader.fileReader(file) # load in the classs object
    processedEvents = 0 # Initialisation
    noiseStart = 0
    firstNoise = False
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    print(len(initial_event_chunk[0]))
    originTime = initial_event_chunk[0].tdcEvents[0].time
    
    

IndexError: list index out of range